In [1]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
# from datatools.analyzer import clean_text
from error_tools import *

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [4]:
root = "/".join(os.getcwd().split("/")[:-1])

path = root + '/error_category_classification/dbdc5_ja_dev_labeled/'
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']



In [5]:
df = read_json_with_NoErr(path, datalist)
print(df.shape)

(2000, 5)


In [6]:
usr_list = []
for d, u, s, ec in zip(df.did, df.usr, df.sys, df.ec):
    usr_list.append(clean_text(u))

In [7]:
# import csv
# filename = "../../corpus/case_frame/data_augment.csv"
# text_list = []
# with open(filename, "r") as f:
#     texts = csv.reader(f)
#     for t in texts:
#         text_list.append(clean_text(t[0]))

In [8]:
text_list = usr_list
pivot = len(text_list)*8//10
X_train_str = text_list[:pivot]
X_test_str = text_list[pivot:]

In [9]:
error_pivot = len(X_train_str)//2
print(error_pivot)
X_no_error = X_train_str[:error_pivot]
X_error = X_train_str[error_pivot:]

800


In [10]:
option = "pos"
seq_len = 4

In [11]:
if option=="pos":
    word_dict = get_all_pos_dict()
    vocab_list = list(pos_dict )

In [12]:
# def fill_SYMBOL(L):
#     return list(map(filler_func, L))
# filler_func = lambda L: ["FOS", *L, "EOS"]
if option == "pos":
    pos = fill_SYMBOL( sentence2pos(X_train_str[:10]) ) 
    sen_id_list = [doc2number(x, word_dict) for x in pos]
    X_train = []
    y_train = []
    for sen_id in sen_id_list:
        splited, targets = split_sequence(sen_id, 4)
        X_train.extend(splited)
        y_train.extend(targets)

In [13]:
leng = len(y_train)
print(leng)
for i, v in enumerate(y_train):
    if leng %(i+1) == 0:
        print(i+1, end=", ")

50
1, 2, 5, 10, 25, 50, 

In [14]:
class Datasets(torch.utils.data.Dataset):
    def __init__(self, X_data, y_data):
        # self.transform = transform

        self.X_data = X_data
        self.y_data = y_data

        self.datanum = len(X_data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_X = self.X_data[idx]
        out_y = self.y_data[idx]

        return out_X, out_y

In [15]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size, batch_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。    
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim,  padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        # LSTMの出力を受け取って全結合してsoftmaxに食わせるための１層のネットワーク
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        self.softmax = nn.LogSoftmax()
    
    def forward(self, x):
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        embeds = self.word_embeddings(x)
        # batch_size, seq_len = x.shape[0], x.shape[1]
        # _, lstm_out = self.lstm(embeds.view(len(x), 1, -1))
        _, lstm_out = self.lstm(embeds)
        # print(hidden_layer)
        # bilstm_out = torch.cat([lstm_out[0][0], lstm_out[0][1]], dim=1)
        tag_space = self.hidden2tag(lstm_out[0])
        # y = self.hidden2tag(hidden_layer[0].view(batch_size, -1))

        # y = self.hidden2tag(bilstm_out)
        y =self.softmax(tag_space.squeeze())
        return y

In [16]:
BATCH_SIZE = 10
epoch_ = 300
trainset = Datasets(X_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

In [17]:
Vocab_size = len(word_dict)
EMBEDDING_DIM = 100
HIDDEN_DIM = EMBEDDING_DIM//2
OUTPUT_DIM = Vocab_size

In [18]:
model = LSTMClassifier(Vocab_size, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, BATCH_SIZE)
if torch.cuda.is_available():
   model.cuda()
# loss_function = nn.NLLLoss()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

In [19]:
losses = []
loss_border = 0.0001
# print("error[{0}]".format(error_types[error_i]))
for epoch in range(epoch_):  # again, normally you would NOT do 300 epochs, it is toy data
    all_loss = 0
    for data in trainloader:
        X_t_tensor = torch.tensor(data[0], device='cuda:0', dtype=torch.long)
        # print(X_t_tensor.shape , y_t_tensor.view(-1,1).shape)
        # y_t_tensor = torch.tensor(data[1].reshape(batch_size, 1), device='cuda:0').float()
        y_t_tensor = torch.tensor(data[1], device='cuda:0', dtype=torch.long)
        optimizer.zero_grad()
        model.zero_grad()
        # print(X_t_tensor.shape)

        score = model(X_t_tensor)
        # print(X_t_tensor.shape, score.view(-1,5).shape, y_t_tensor.view(-1,1).shape)
        loss_ = loss_function(score,  y_t_tensor)
        loss_.backward()
        all_loss += loss_.item()
        optimizer.step()
        del score
        del loss_
    losses.append(all_loss)
    if (epoch+1) % 50 == 0:
        print("epoch", epoch+1, "\t" , "loss", all_loss)
    # if all_loss <= loss_border:
    #     print("loss was under border(={0}) : train end".format(loss_border))
    #     break
print("done")

/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 50 	 loss 0.20076712407171726
epoch 100 	 loss 0.053571360651403666
epoch 150 	 loss 0.02621308946982026
epoch 200 	 loss 0.015683723147958517
epoch 250 	 loss 0.010536902118474245
epoch 300 	 loss 0.007578925346024334
done


In [20]:
if option == "pos":
    pos = fill_SYMBOL( sentence2pos(X_test_str) ) 
    sen_id_list = [doc2number(x, word_dict) for x in pos]
    X_test = []
    y_test = []
    for sen_id in sen_id_list:
        splited, targets = split_sequence(sen_id, 4)
        X_test.extend(splited)
        y_test.extend(targets)

In [23]:
with torch.no_grad():
    X_tensor = torch.tensor(X_test, dtype=torch.long, device='cuda:0')
    y_tensor = torch.tensor(y_test, dtype=torch.long, device='cuda:0')
            # 推論
    y_pred= np.array(model(X_tensor).cpu()).argmax(axis=1)

/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [24]:
metrics.accuracy_score(y_test, y_pred)

0.44232804232804235